In [ ]:
import pandas as pd
import numpy as np

In [2]:
robbins_craters = pd.read_csv("./lunar_crater_database_robbins_2018.csv")
yang_aged_craters = pd.read_csv("./Aged_Lunar_Crater_Database_DeepCraters_2020.csv")

The Yang 2020 database contains the ages for all craters larger than 10km in diameter. We need to process the Robbins 2018 database to find all craters less than 10km.

In [3]:
lt_10_inds = robbins_craters["DIAM_CIRC_IMG"] < 10
robbins_craters = robbins_craters[lt_10_inds]

For the sake of making modular arithmatic easier, we will add 360 to all negative longitudes in both databases.

In [4]:
def correct_lon_column(df, lon_name):
    """
    Add 360 to all negative longitude values in dataframe.

    Has the side effect of modifying the passed in dataframe.

    df: Pandas dataframe
    name: Name of column in the dataframe
    """
    def bool_to_deg(x):
        if x:
            return 360
        return 0
    neg_lon = df[lon_name] < 0
    lon_correct_factor = [bool_to_deg(i) for i in neg_lon]
    df[lon_name] = df[lon_name] + lon_correct_factor

correct_lon_column(robbins_craters, "LON_CIRC_IMG")
correct_lon_column(yang_aged_craters, "Lon")

We will now look at every crater smaller than 10km in Robbins 2018, and see if it lies within Yang 2020.

To check if a child crater lies within a parent crater, we first calculate the distance between the centerpoints of each crater. Then, we check if that distance is smaller the radius of the parent crater.

In [6]:
# https://nssdc.gsfc.nasa.gov/planetary/factsheet/moonfact.html
MOON_MEAN_RADIUS = 1737.4 # km


def great_cirlce_distance(a_lon, a_lat, b_lon, b_lat, radius=MOON_MEAN_RADIUS):
    """
    Calculate the great circle distance between two points.

    Assumes lon/lat is passed in degrees.
    """
    a_lon = np.deg2rad(a_lon)
    a_lat = np.deg2rad(a_lat)
    b_lon = np.deg2rad(b_lon)
    b_lat = np.deg2rad(b_lat)

    angle = np.arccos(np.sin(a_lat) * np.sin(b_lat)
                      + np.cos(a_lat) * np.cos(b_lat) * np.cos(a_lon-b_lon))
    return angle * radius


def crater_in_crater(c_lon, c_lat, p_lon, p_lat, p_diam):
    """
    Check if child crater c lies within parent crater p.
    """
    dist = great_cirlce_distance(c_lon, c_lat, p_lon, p_lat)
    return dist < p_diam


def child_in_parents()


def craters_in_craters(child_df, parent_df):
    """
    Check if craters in child_df are in parent_df.

    Both input dataframes have the columns: ["id", "lon", "lat", "diam"]

    Has the side effect of adding a new column to parent_df called "children_ids",
    the set of IDs of child craters.
    """
    pass

In [7]:
yang_aged_craters["Child_Robbins_Crater_IDs"] = [set() for _ in range(len(yang_aged_craters))]

for robbins_ind, robbins_crater in robbins_craters.iterrows():
    robbins_lon  = robbins_crater["LON_CIRC_IMG"]
    robbins_lat  = robbins_crater["LAT_CIRC_IMG"]
    
    for yang_ind, yang_crater in yang_aged_craters.iterrows():
        yang_lon  = yang_crater["Lon"]
        yang_lat  = yang_crater["Lat"]
        yang_diam = yang_crater["Diam_km"]
        if crater_in_crater(robbins_lon, robbins_lat, yang_lon, yang_lat, yang_diam):
            robbins_id = robbins_craters["CRATER_ID"][robbins_ind]
            yang_aged_craters["Child_Robbins_Crater_IDs"][yang_ind].add(robbins_id)
            break

/tmp/ipykernel_8911/2318420125.py:16: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(np.sin(a_lat) * np.sin(b_lat)


KeyboardInterrupt: 

In [12]:
yang_aged_craters

,Flags_data,ID,Lat,Lon,Diam_km,Age,Child_Robbins_Crater_IDs
0,CE1,1,-8.503370,293.189000,19.987530,2,{}
1,CE1,2,-7.408700,8.209640,19.986876,5,{}
2,CE1,3,64.340500,67.728600,19.985744,5,{}
3,CE1,4,-32.517300,34.820200,19.985460,2,{}
4,CE1,5,21.586500,80.999700,19.984877,2,{}
...,...,...,...,...,...,...,...
18991,CE2,116980,-63.872833,119.773304,10.584219,1,{}
18992,CE2,117003,-64.415138,180.715665,11.905659,2,{}
18993,CE2,117017,-64.372227,203.861354,14.643919,2,{}
18994,CE2,117238,-64.545580,174.082361,8.129036,5,{}
